# 🔬 Problème de Poisson — Régression avec CNN
Ce notebook génère un dataset synthétique basé sur l'équation de Poisson, puis entraîne un modèle CNN pour approximer la solution $u(x, y)$ à partir de $f(x, y)$.

In [ ]:
# 📦 Imports
import numpy as np
import matplotlib.pyplot as plt
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
from scipy.sparse import diags, kron
from scipy.sparse.linalg import spsolve
from tqdm import tqdm
import random

In [ ]:
# ⚙️ Génération du dataset de Poisson 2D
def generate_rhs(nx, ny, a, b):
    x = np.linspace(0, 1, nx)
    y = np.linspace(0, 1, ny)
    X, Y = np.meshgrid(x, y, indexing='ij')
    return X * np.sin(a * np.pi * Y) + Y * np.sin(b * np.pi * X)

def solve_poisson(f, h):
    n = f.shape[0]
    main = -4 * np.ones(n)
    off = np.ones(n - 1)
    T = diags([main, off, off], [0, -1, 1])
    I = diags([np.ones(n)], [0])
    A = (kron(I, T) + kron(T, I)) / (h ** 2)
    f_flat = f[1:-1, 1:-1].flatten()
    u_flat = spsolve(A, f_flat)
    u = np.zeros_like(f)
    u[1:-1, 1:-1] = u_flat.reshape((n - 2, n - 2))
    return u

In [ ]:
class PoissonDataset(Dataset):
    def __init__(self, n_samples=1000, size=64):
        self.data = []
        h = 1 / (size - 1)
        for _ in tqdm(range(n_samples)):
            a, b = random.uniform(1, 5), random.uniform(1, 5)
            f = generate_rhs(size, size, a, b)
            u = solve_poisson(f, h)
            self.data.append((f.astype(np.float32), u.astype(np.float32)))
    def __len__(self):
        return len(self.data)
    def __getitem__(self, idx):
        f, u = self.data[idx]
        return torch.tensor(f).unsqueeze(0), torch.tensor(u).unsqueeze(0)

In [ ]:
# 🧠 Modèle CNN simple
class CNNPoisson(nn.Module):
    def __init__(self):
        super().__init__()
        self.net = nn.Sequential(
            nn.Conv2d(1, 16, 3, padding=1), nn.ReLU(),
            nn.Conv2d(16, 32, 3, padding=1), nn.ReLU(),
            nn.Conv2d(32, 16, 3, padding=1), nn.ReLU(),
            nn.Conv2d(16, 1, 3, padding=1)
        )
    def forward(self, x):
        return self.net(x)

In [ ]:
# 🚂 Entraînement
dataset = PoissonDataset(n_samples=500, size=64)
train_loader = DataLoader(dataset, batch_size=8, shuffle=True)

model = CNNPoisson()
criterion = nn.MSELoss()
optimizer = optim.Adam(model.parameters(), lr=1e-3)

for epoch in range(10):
    losses = []
    for f, u in train_loader:
        pred = model(f)
        loss = criterion(pred, u)
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        losses.append(loss.item())
    print(f"Epoch {epoch+1}: Loss = {np.mean(losses):.6f}")

In [ ]:
# 📊 Visualisation d’un exemple
f_sample, u_true = dataset[0]
u_pred = model(f_sample.unsqueeze(0)).detach().squeeze().numpy()

fig, axs = plt.subplots(1, 3, figsize=(15, 4))
axs[0].imshow(f_sample.squeeze(), cmap='viridis')
axs[0].set_title('f(x, y)')
axs[1].imshow(u_true.squeeze(), cmap='viridis')
axs[1].set_title('u vrai')
axs[2].imshow(u_pred, cmap='viridis')
axs[2].set_title('u prédit')
plt.tight_layout()
plt.show()